## Import Basic Libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn 

## Import Data

In [3]:
# training data
train = pd.read_csv('/content/test.csv')
test = pd.read_csv('/content/train.csv')

## Exploratory Data Analysis

In [4]:
train.head(10)

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."
5,31968,choose to be :) #momtips
6,31969,something inside me dies ð¦ð¿â¨ eyes nes...
7,31970,#finished#tattoo#inked#ink#loveitâ¤ï¸ #â¤ï¸...
8,31971,@user @user @user i will never understand why...
9,31972,#delicious #food #lovelife #capetown mannaep...


In [5]:
test.tail()

,id,label,tweet
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."
31961,31962,0,thank you @user for you follow


In [9]:
# non-racist/sexist related tweets

sum(test['label'] == 0)

29720

In [11]:
# racist/sexist related tweets
sum(test['label'] == 1)

2242

In [12]:
# check if there are any missing values
train.isnull().sum()

id       0
tweet    0
dtype: int64

## Data Cleaning

In [14]:
#install twitter-preprocessor to clean the tweets
!pip install tweet-preprocessor --upgrade --quiet

In [15]:
# remove special characters using the regular expression library
import re

# setup punctuations we want to be replaced
REPLACE_NO_SPACE =  re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [17]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [18]:
# clean training data
train_tweet = clean_tweets(train["tweet"])
train_tweet = pd.DataFrame(train_tweet)

In [19]:
# append cleaned tweets to the training data
train["clean_tweet"] = train_tweet

# compare the cleaned and uncleaned tweets
train.head(10)

,id,tweet,clean_tweet
0,31963,#studiolife #aislife #requires #passion #dedic...,to find
1,31964,@user #white #supremacists want everyone to s...,want everyone to see the new and heres why
2,31965,safe ways to heal your #acne!! #altwaystohe...,safe ways to heal your
3,31966,is the hp and the cursed child book up for res...,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",rd to my amazing hilarious eli ahmir uncle dav...
5,31968,choose to be :) #momtips,choose to be
6,31969,something inside me dies ð¦ð¿â¨ eyes nes...,something inside me dies eyes ness
7,31970,#finished#tattoo#inked#ink#loveitâ¤ï¸ #â¤ï¸...,
8,31971,@user @user @user i will never understand why...,i will never understand why my dad left me whe...
9,31972,#delicious #food #lovelife #capetown mannaep...,mannaepicure


In [20]:
# clean the test data and append the cleaned tweets to the test data
test_tweet = clean_tweets(test["tweet"])
test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
test["clean_tweet"] = test_tweet

# compare the cleaned and uncleaned tweets
test.tail()

,id,label,tweet,clean_tweet
31957,31958,0,ate @user isz that youuu?ðððððð...,ate isz that youuu
31958,31959,0,to see nina turner on the airwaves trying to...,to see nina turner on the airwaves trying to w...
31959,31960,0,listening to sad songs on a monday morning otw...,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...",vandalised in in condemns act
31961,31962,0,thank you @user for you follow,thank you for you follow


## Test and Train split

In [23]:

from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = test.label.values

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(test.clean_tweet.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

## Vectorize tweets using CountVectorizer

#### CountVectorizer Example



In [24]:
from sklearn.feature_extraction.text import CountVectorizer


In [27]:
documents=["Reinforcement learning is one of three basic machine learning paradigms",
           "alongside supervised learning and unsupervised learning"]
# initializing the countvectorizer
vectorizer = CountVectorizer()

# tokenize and make the document into a matrix
document_term_matrix = vectorizer.fit_transform(documents)

# check the result
pd.DataFrame(document_term_matrix.toarray(), columns = vectorizer.get_feature_names())

,alongside,and,basic,is,learning,machine,of,one,paradigms,reinforcement,supervised,three,unsupervised
0,0,0,1,1,2,1,1,1,1,1,0,1,0
1,1,1,0,0,2,0,0,0,0,0,1,0,1


In [28]:

from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

## Model Building

### Apply Support vector classifier

In [29]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

## Accuracy score for SVC¶


In [30]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')


Accuracy score for SVC is:  94.86912086766085 %
